In [8]:
import urllib
from zipfile import ZipFile
from io import StringIO
import shapefile
import geopandas as gpd
from shapely.geometry import shape  
import osr
import pandas as pd
import requests
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
county_file_url = 'http://www2.census.gov/geo/tiger/GENZ2015/shp/cb_2015_us_county_500k.zip'

zipfile = ZipFile(StringIO(urllib.request.urlopen(county_file_url).read()))

TypeError: initial_value must be str or None, not bytes